<a href="https://colab.research.google.com/github/junsookim1/Everday_Practice/blob/master/Koyang_Bike_Data(%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%85%98_%EA%B1%B0%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import missingno as msno


In [ ]:
# pip install haversine

In [ ]:
from haversine import haversine

In [ ]:
df_OH=pd.read_csv('01.운영이력.csv')
df_Bike=pd.read_csv('02.자전거스테이션.csv')

/home/jun/anaconda3/envs/jun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_Bike

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530
...,...,...,...,...,...
159,350,★하이파크5단지 502동앞 버스정류장,20,37.697867,126.753089
160,351,◆일산에듀포레 푸르지오,40,37.702259,126.767231
161,352,◆꿈에그린203동앞,40,37.666425,126.749244
162,353,◆꿈에그린106동앞,40,37.666720,126.750784


In [ ]:
df_Bike['위도'][0]

37.65477464

In [ ]:
kyobo = (df_Bike['위도'][2], df_Bike['경도'][2]) # 교보 문고 위도/경도
gangnam = (df_Bike['위도'][163], df_Bike['경도'][163]) # 강남역 위도 경도

haversine(kyobo, gangnam) *1000

9029.194586031936

In [ ]:
# 거리 구하기...일단 for문은 완성인데 어떻게 하면 편하게 돌릴 수 있는지 확인하기 내 생각대로!!
station_1 = []
station_2 = []
harv = []
for sta, lat, long in zip(df_Bike['Station_ID'],df_Bike['위도'],df_Bike['경도']):
    for sta1, lat1, long1 in zip(df_Bike['Station_ID'],df_Bike['위도'],df_Bike['경도']):
        x = (lat, long)
        y = (lat1, long1) 
        h = haversine(x, y) *1000
        station_1.append(sta)
        station_2.append(sta1)
        harv.append(h)


In [ ]:
df_sta_1 = pd.DataFrame(station_1, columns=['sta_rent'])
df_sta_2 = pd.DataFrame(station_2, columns=['sta_return'])
df_harv = pd.DataFrame(harv, columns=['이동거리'])

In [ ]:
df_distance = pd.concat([df_sta_1,df_sta_2, df_harv], axis = 1)
df_distance

,sta_rent,sta_return,이동거리
0,101,101,0.000000
1,101,103,810.593473
2,101,104,7749.401651
3,101,105,414.953376
4,101,106,715.798911
...,...,...,...
26891,992,350,9768.976190
26892,992,351,9278.739241
26893,992,352,8104.699800
26894,992,353,7993.663782


In [ ]:
df_distance.to_csv('스테이션 별 이동거리.csv') # 저장
df_distance = pd.read_csv('스테이션 별 이동거리.csv')# 불러오기

ㄴ# a.merge(b, on=['sta_rent','sta_return']) - merge하기

# station 거리
- 대여 스테이션과 반납 스테이션이 같을 경우 다들 경우를 구해서 데이터의 의미를 찾아보기

In [ ]:
df_OH=pd.read_csv('01.운영이력_ver1(한글컬럼, 이용거리).csv') # 기존 데이터

/home/jun/anaconda3/envs/jun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 대여 스테이션 번호 != 반납 스테이션 번호
- 추정 이동거리가 0과 0이 아닌경우 구하고
- 그 중에서 이용시간이 0과 0이 아닌 경우 같이 구한다.
- df1 : 추정 이동거리가 != 0
- df2 : 추정 이동거리가 == 0

In [ ]:
df_OH

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0


In [ ]:
df_nonsame_station_dis = df_OH[df_OH['대여 스테이션 번호'] != df_OH['반납 스테이션 번호']]
df_nonsame_station_dis

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
5,15945546,2017-01-01 00:03:40,309,8,2017-01-01 00:10:50,210,3,0.0,99,0,3185081.0,1A84400000F505,NaN,430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0


In [ ]:
df_nonsame_dis = df_nonsame_station_dis.loc[:,['대여 스테이션 번호','반납 스테이션 번호','추정 이동 거리','이용 시간(s)']]
df_nonsame_dis

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
0,213,260,0.0,791.0
2,119,0,NaN,3600.0
3,121,133,15490.0,829.0
4,320,259,0.0,912.0
5,309,210,0.0,430.0
...,...,...,...,...
3142683,222,219,0.0,377.0
3142684,343,311,0.0,240.0
3142685,333,322,0.0,657.0
3142686,264,210,0.0,685.0


In [ ]:
#추정 이동거리가 0이 아닐때 데이터
df_1 = df_nonsame_dis[df_nonsame_dis['추정 이동 거리'] != 0]
df_1

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
2,119,0,NaN,3600.0
3,121,133,15490.0,829.0
6,232,213,8948.0,541.0
8,114,113,17482.0,164.0
11,215,231,15170.0,852.0
...,...,...,...,...
3142668,245,226,38894.0,712.0
3142673,252,218,37720.0,864.0
3142679,211,0,NaN,3386.0
3142680,228,0,NaN,3410.0


In [ ]:
df_1[df_1['이용 시간(s)'] ==0] # 이용시간 0과 0이 아닌 경우도 같이 구해 준다.

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
9824,220,0,NaN,0.0
11008,114,0,NaN,0.0
35944,317,0,NaN,0.0
36603,321,0,NaN,0.0
41916,309,228,40508.0,0.0
...,...,...,...,...
3107640,203,318,41250.0,0.0
3128301,259,103,15441.0,0.0
3139024,212,231,11317.0,0.0
3142052,341,0,NaN,0.0


In [ ]:
df_1[df_1['이용 시간(s)'] !=0]

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
2,119,0,NaN,3600.0
3,121,133,15490.0,829.0
6,232,213,8948.0,541.0
8,114,113,17482.0,164.0
11,215,231,15170.0,852.0
...,...,...,...,...
3142668,245,226,38894.0,712.0
3142673,252,218,37720.0,864.0
3142679,211,0,NaN,3386.0
3142680,228,0,NaN,3410.0


In [ ]:
#추정 이동거리가 0일때의 이용 시간
df_2 = df_nonsame_dis[df_nonsame_dis['추정 이동 거리'] == 0] 
df_2

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
0,213,260,0.0,791.0
4,320,259,0.0,912.0
5,309,210,0.0,430.0
7,307,305,0.0,227.0
9,231,210,0.0,613.0
...,...,...,...,...
3142682,210,316,0.0,1874.0
3142683,222,219,0.0,377.0
3142684,343,311,0.0,240.0
3142685,333,322,0.0,657.0


In [ ]:
df_2[df_2['이용 시간(s)'] ==0] # 이용시간 0과 0이 아닌 경우도 같이 구해 준다.

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
129010,165,106,0.0,0.0
195637,260,304,0.0,0.0
215863,339,343,0.0,0.0
230164,318,343,0.0,0.0
233143,128,116,0.0,0.0
...,...,...,...,...
3120691,235,231,0.0,0.0
3131903,306,264,0.0,0.0
3132241,211,265,0.0,0.0
3133135,317,225,0.0,0.0


In [ ]:
df_2[df_2['이용 시간(s)'] !=0]

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
0,213,260,0.0,791.0
4,320,259,0.0,912.0
5,309,210,0.0,430.0
7,307,305,0.0,227.0
9,231,210,0.0,613.0
...,...,...,...,...
3142682,210,316,0.0,1874.0
3142683,222,219,0.0,377.0
3142684,343,311,0.0,240.0
3142685,333,322,0.0,657.0


### 대여 스테이션 번호 == 반납 스테이션 번호
- 추정 이동거리가 0과 0이 아닌경우 구하고
- 그 중에서 이용시간이 0과 0이 아닌 경우 같이 구한다.
- df3 : 추정 이동거리가 != 0
- df4 : 추정 이동거리가 == 0

In [ ]:
df_same_station_dis = df_OH[df_OH['대여 스테이션 번호'] == df_OH['반납 스테이션 번호']] 
df_same_station_dis
# 대여와 반납이 스테이션이 같다면 

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
10,15945552,2017-01-01 00:13:50,215,17,2017-01-01 00:14:33,215,17,0.0,2,187342,0.0,1A84400000F568,NaN,43.0
20,15945565,2017-01-01 00:23:30,225,12,2017-01-01 00:23:30,225,12,65535.0,99,0,3185089.0,1A84400000F692,NaN,0.0
55,15945607,2017-01-01 00:59:13,333,12,2017-01-01 01:45:17,333,3,5860.0,2,187678,0.0,1A84400000F69E,NaN,2764.0
62,15945616,2017-01-01 01:02:38,319,29,2017-01-01 01:15:25,319,19,10634.0,3,187846,0.0,1A8440000007A0,NaN,767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142578,21718464,2019-12-31 21:14:37,346,11,2019-12-31 21:34:01,346,13,0.0,3,237062,0.0,1A844000000772,NaN,1164.0
3142586,21718484,2019-12-31 21:31:07,161,9,2019-12-31 22:00:06,161,17,0.0,99,0,4124646.0,1A844000000046,NaN,1739.0
3142599,21718500,2019-12-31 21:47:37,309,14,2019-12-31 21:47:51,309,14,0.0,24,76408,0.0,1A84400000FB04,NaN,14.0
3142627,21718533,2019-12-31 22:33:31,235,10,2019-12-31 22:33:58,235,11,0.0,99,0,4124653.0,1A84400000F9B6,NaN,27.0


In [ ]:
df_same_station_dis['대여 스테이션 번호'].unique() # 기존의 스테이션 번호 분석과 비교 해보기!

array([231, 215, 225, 333, 319, 118, 245, 210, 232, 307, 143, 252, 233,
       203, 328, 342, 340, 317, 318, 309, 324, 314, 306, 218, 248, 211,
       259, 311, 254, 253, 165, 258, 228, 349, 201, 131, 207, 223, 167,
       234, 343, 331, 339, 222, 166, 260, 323, 312, 249, 213, 111, 320,
       172, 105, 227, 164, 104, 141, 101, 345, 202, 305, 221, 347, 224,
       250, 315, 304, 330, 346, 229, 137, 205, 217, 206, 316, 169, 142,
       257, 115, 308, 113, 313, 321, 226, 329, 214, 303, 138, 162, 246,
       230, 235, 240, 119, 130, 216, 327, 140, 171, 173, 301, 129, 163,
       239, 204, 144, 350, 116, 212, 236, 237, 322, 112, 127, 326, 247,
       168, 103, 106, 114, 334, 302, 128, 992, 170, 256, 310, 341, 219,
       121, 146, 238, 251, 161, 123, 110, 148, 220, 255, 139, 348, 124,
       126, 125, 133, 176, 177, 178, 261, 262, 351, 265, 264, 263, 998])

In [ ]:
df_OH_dis = df_same_station_dis.loc[:,['대여 스테이션 번호','반납 스테이션 번호','추정 이동 거리','이용 시간(s)']]
df_OH_dis 

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
1,231,231,31039.0,2961.0
10,215,215,0.0,43.0
20,225,225,65535.0,0.0
55,333,333,5860.0,2764.0
62,319,319,10634.0,767.0
...,...,...,...,...
3142578,346,346,0.0,1164.0
3142586,161,161,0.0,1739.0
3142599,309,309,0.0,14.0
3142627,235,235,0.0,27.0


In [ ]:
#추정 이동거리가 0이 아닐때 데이터
df_3 = df_OH_dis[df_OH_dis['추정 이동 거리'] != 0]
df_3

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
1,231,231,31039.0,2961.0
20,225,225,65535.0,0.0
55,333,333,5860.0,2764.0
62,319,319,10634.0,767.0
131,245,245,2.0,11.0
...,...,...,...,...
3142276,230,230,30.0,51.0
3142309,162,162,11750.0,862.0
3142515,302,302,21550.0,2168.0
3142524,238,238,18123.0,24.0


In [ ]:
df_3[df_3['이용 시간(s)'] == 0]

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
20,225,225,65535.0,0.0
3191,322,322,17767.0,0.0
4644,306,306,30318.0,0.0
6060,303,303,42645.0,0.0
11832,302,302,9039.0,0.0
...,...,...,...,...
3136770,233,233,66.0,0.0
3138840,231,231,11317.0,0.0
3139454,340,340,6839.0,0.0
3140589,306,306,59.0,0.0


In [ ]:
df_3[df_3['이용 시간(s)'] != 0]

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
1,231,231,31039.0,2961.0
55,333,333,5860.0,2764.0
62,319,319,10634.0,767.0
131,245,245,2.0,11.0
149,232,232,4582.0,2.0
...,...,...,...,...
3142276,230,230,30.0,51.0
3142309,162,162,11750.0,862.0
3142515,302,302,21550.0,2168.0
3142524,238,238,18123.0,24.0


In [ ]:
#추정 이동거리가 0이 아닐때 데이터
df_4 = df_OH_dis[df_OH_dis['추정 이동 거리'] == 0]
df_4

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
10,215,215,0.0,43.0
90,118,118,0.0,15.0
134,210,210,0.0,48.0
138,210,210,0.0,133.0
272,252,252,0.0,21.0
...,...,...,...,...
3142578,346,346,0.0,1164.0
3142586,161,161,0.0,1739.0
3142599,309,309,0.0,14.0
3142627,235,235,0.0,27.0


In [ ]:
df_4[df_4['이용 시간(s)'] == 0]

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
2198,323,323,0.0,0.0
3190,260,260,0.0,0.0
5963,207,207,0.0,0.0
10164,225,225,0.0,0.0
18133,218,218,0.0,0.0
...,...,...,...,...
3138725,228,228,0.0,0.0
3139682,253,253,0.0,0.0
3140088,345,345,0.0,0.0
3141491,235,235,0.0,0.0


In [ ]:
df_4[df_4['이용 시간(s)'] != 0]

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,이용 시간(s)
10,215,215,0.0,43.0
90,118,118,0.0,15.0
134,210,210,0.0,48.0
138,210,210,0.0,133.0
272,252,252,0.0,21.0
...,...,...,...,...
3142578,346,346,0.0,1164.0
3142586,161,161,0.0,1739.0
3142599,309,309,0.0,14.0
3142627,235,235,0.0,27.0
